# *Gerar embeddings de 1000 fotos*#


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
IMAGES_DIR = "/content/drive/MyDrive/Trabalho-ED/lfw_funneled"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.3 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=f9438b4ee9410ec6589f39ac6c0bd9fa740b0bc3d2dcd2762899ea5937de4d77
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [ ]:
from deepface import DeepFace
import os
from PIL import Image

In [ ]:
IMAGES_DIR = "/content/drive/MyDrive/Trabalho-ED/lfw_funneled"
extensoes_permitidas = {".jpg", ".jpeg", ".png", ".bmp"}

In [ ]:
import os
from deepface import DeepFace

# 1) Defina a pasta raiz onde estão as imagens
IMAGES_DIR = "/content/drive/MyDrive/Trabalho-ED/lfw_funneled/"

# 2) Extensões válidas
extensoes_permitidas = {".jpg", ".jpeg", ".png", ".bmp"}

# 3) Busca recursiva pelas imagens nas subpastas
imagens = []
for raiz, pastas, arquivos in os.walk(IMAGES_DIR):
    for nome_arquivo in arquivos:
        ext = os.path.splitext(nome_arquivo)[1].lower()
        if ext in extensoes_permitidas:
            caminho_completo = os.path.join(raiz, nome_arquivo)
            imagens.append(caminho_completo)

print(f"Total de arquivos de imagem encontrados (antes de limitar): {len(imagens)}")

# 4) Limita às primeiras N imagens
N = 1000 # ou 1000 se quiser tudo
# imagens = imagens[:N]
print(f"Usando apenas as primeiras {N} imagens para gerar embeddings.\n")

if len(imagens) == 0:
    raise ValueError(f"Nenhuma imagem válida encontrada em {IMAGES_DIR}")

# 5) Inicializa estruturas
embeddings_dict = {}      # { nome_pessoa: [embedding] }
imagens_sem_rosto = []    # caminhos das imagens que falharam

# 6) Loop principal
for caminho_img in imagens:
    try:
        # Extrai o nome da pessoa (pasta onde a imagem está)
        nome_pessoa = os.path.basename(os.path.dirname(caminho_img))

        resultado = DeepFace.represent(
            img_path=caminho_img,
            model_name='Facenet',
            enforce_detection=False
        )
        vetor = resultado[0]['embedding']

        embeddings_dict[nome_pessoa] = vetor
        # print(len(embeddings_dict))
        if len(embeddings_dict) == 1000:
           break

    except Exception as e:
        imagens_sem_rosto.append(caminho_img)
        print(f"[ERRO] Falha ao processar {caminho_img}: {str(e)}")

# 7) Relatórios
print(f"\n✅ Total de embeddings gerados com sucesso: {len(embeddings_dict)}")
print(f"❌ Total de imagens puladas (sem rosto detectado): {len(imagens_sem_rosto)}")

if imagens_sem_rosto:
    print("\nImagens que falharam na detecção:")
    for img in imagens_sem_rosto:
        print("  -", img)


Total de arquivos de imagem encontrados (antes de limitar): 13233
Usando apenas as primeiras 1000 imagens para gerar embeddings.


✅ Total de embeddings gerados com sucesso: 1000
❌ Total de imagens puladas (sem rosto detectado): 0


In [ ]:
import json

# 8) Prepara uma lista de dicionários para salvar em JSON
lista_para_json = []
print(len(embeddings_dict))
for nome, vetor in embeddings_dict.items():
    entrada = {
        "id": nome,         # Nome da pessoa
        "embedding": vetor  # Vetor de 128 floats
    }
    lista_para_json.append(entrada)

# 9) Salva no arquivo .json
caminho_arquivo_json = "/content/drive/MyDrive/Trabalho-ED/embeddings.json"
with open(caminho_arquivo_json, "w") as f:
    json.dump(lista_para_json, f, indent=2)

print(f"\n📁 Embeddings salvos em: {caminho_arquivo_json}")


1000

📁 Embeddings salvos em: /content/drive/MyDrive/Trabalho-ED/embeddings.json


Bloco de busca com Ngrok para o FastApi: OBS precisa utilizar outras ferramentas para serem executadas


In [ ]:
api_url = " https://4371-2804-14c-7582-85f3-11ed-4417-1dbc-6146.ngrok-free.app/buscar"
payload = {"embedding": embedding}
response = requests.post(api_url, json=payload)

if response.status_code == 200:
    print("Success!")
else:
    print("Error:", response.status_code)